In [11]:
import time
from typing import Any, Optional, Dict
from collections import OrderedDict
import threading

class SimpleCache:
    """
    Enhanced cache with TTL, LRU eviction, statistics, and thread safety.
    """

    def __init__(self, max_size: int = 100, default_ttl: Optional[int] = None):
        """
        Initialize cache with size limit and default TTL.

        Args:
            max_size: Maximum number of items to store
            default_ttl: Default time-to-live in seconds (None = no expiration)
        """
        self.max_size = max_size
        self.default_ttl = default_ttl

        # Data structures
        self._data = OrderedDict()  # key -> value
        self._expiry_times = {}     # key -> expiry timestamp
        self._access_times = {}     # key -> last access timestamp

        # Statistics
        self._stats = {
            'hits': 0,
            'misses': 0,
            'evictions': 0,
            'insertions': 0,
            'expirations': 0
        }

        # Thread safety
        self._lock = threading.RLock()

    def get(self, key: str) -> Optional[Any]:
        """
        Get value from cache with TTL and LRU support.

        Args:
            key: Cache key

        Returns:
            Cached value or None if not found/expired
        """
        with self._lock:
            # Check if key exists and not expired
            if not self._has_key(key):
                self._stats['misses'] += 1
                return None

            # Update access time and move to end (most recently used)
            self._access_times[key] = time.time()
            self._data.move_to_end(key)

            self._stats['hits'] += 1
            return self._data[key]

    def set(self, key: str, value: Any, ttl: Optional[int] = None) -> None:
        """
        Set value in cache with TTL and LRU eviction.

        Args:
            key: Cache key
            value: Value to cache
            ttl: Time-to-live in seconds (overrides default)
        """
        with self._lock:
            # Evict if needed (before adding new item)
            if len(self._data) >= self.max_size and key not in self._data:
                self._evict_lru(1)

            # Store data
            self._data[key] = value
            self._access_times[key] = time.time()

            # Set expiry time
            actual_ttl = ttl if ttl is not None else self.default_ttl
            if actual_ttl is not None:
                self._expiry_times[key] = time.time() + actual_ttl
            elif key in self._expiry_times:
                del self._expiry_times[key]  # Remove expiry if TTL is None

            # Move to end (most recently used)
            self._data.move_to_end(key)
            self._stats['insertions'] += 1

    def delete(self, key: str) -> bool:
        """Delete key from cache and all tracking data."""
        with self._lock:
            if key in self._data:
                del self._data[key]
                self._expiry_times.pop(key, None)
                self._access_times.pop(key, None)
                return True
            return False

    def clear(self) -> None:
        """Clear all items from cache."""
        with self._lock:
            self._data.clear()
            self._expiry_times.clear()
            self._access_times.clear()
            # Reset statistics
            self._stats = {k: 0 for k in self._stats}

    def size(self) -> int:
        """Return current number of items in cache."""
        with self._lock:
            return len(self._data)

    def get_stats(self) -> Dict[str, int]:
        """
        Get cache statistics.

        Returns:
            Dict with keys: hits, misses, evictions, current_size
        """
        with self._lock:
            stats = self._stats.copy()
            stats['current_size'] = len(self._data)
            stats['expired_count'] = self._count_expired()
            return stats

    def cleanup_expired(self) -> int:
        """
        Remove expired items from cache.

        Returns:
            Number of items removed
        """
        with self._lock:
            expired_keys = []
            for key in list(self._data.keys()):
                if self._is_expired(key):
                    expired_keys.append(key)

            for key in expired_keys:
                self.delete(key)
                self._stats['expirations'] += 1

            return len(expired_keys)

    def _evict_lru(self, count: int = 1) -> int:
        """
        Evict least recently used items.

        Args:
            count: Number of items to evict

        Returns:
            Number of items actually evicted
        """
        evicted = 0
        while evicted < count and self._data:
            key, _ = self._data.popitem(last=False)  # Remove from beginning (LRU)
            self._expiry_times.pop(key, None)
            self._access_times.pop(key, None)
            self._stats['evictions'] += 1
            evicted += 1
        return evicted

    def _is_expired(self, key: str) -> bool:
        """Check if a cache entry has expired."""
        if key not in self._expiry_times:
            return False
        return time.time() > self._expiry_times[key]

    def _has_key(self, key: str) -> bool:
        """Check if key exists and is not expired."""
        if key not in self._data:
            return False
        if self._is_expired(key):
            self.delete(key)
            return False
        return True

    def _count_expired(self) -> int:
        """Count number of expired items in cache."""
        return sum(1 for key in self._data if self._is_expired(key))

    def get_oldest_key(self) -> Optional[str]:
        """Get the least recently used key (for testing)."""
        with self._lock:
            if self._data:
                return next(iter(self._data))
            return None

    def get_newest_key(self) -> Optional[str]:
        """Get the most recently used key (for testing)."""
        with self._lock:
            if self._data:
                return next(reversed(self._data))
            return None

# Test Enhanced Cache:
# Test the enhanced implementation
if __name__ == "__main__":
    print("=== Testing Enhanced Cache ===")

    # Test TTL functionality
    cache = SimpleCache(max_size=3, default_ttl=1)

    print("=== Testing TTL ===")
    cache.set("temp_key", "temp_value")
    print(f"Immediately after set: {cache.get('temp_key')}")
    time.sleep(1.1)
    print(f"After TTL expired: {cache.get('temp_key')}")

    print("\n=== Testing Size Limits & LRU ===")
    cache.clear()
    cache.set("a", 1, ttl=None)
    cache.set("b", 2, ttl=None)
    cache.set("c", 3, ttl=None)
    print(f"Cache size after adding 3 items: {cache.size()}")

    # Access 'a' to make it recently used
    cache.get("a")
    print(f"Oldest key: {cache.get_oldest_key()}, Newest key: {cache.get_newest_key()}")

    # Add 'd' which should evict 'b' (least recently used)
    cache.set("d", 4, ttl=None)
    print(f"After adding 'd': a={cache.get('a')}, b={cache.get('b')}, c={cache.get('c')}, d={cache.get('d')}")

    print("\n=== Testing Statistics ===")
    stats = cache.get_stats()
    print(f"Cache statistics: {stats}")
    print("\n=== Testing Cleanup ===")
    cache.set("expire_me", "value", ttl=1)
    print(f"Before cleanup - size: {cache.size()}")
    time.sleep(1.1)
    removed_count = cache.cleanup_expired()
print(f"Expired items removed: {removed_count}, After cleanup - size: {cache.size()}")


=== Testing Enhanced Cache ===
=== Testing TTL ===
Immediately after set: temp_value
After TTL expired: None

=== Testing Size Limits & LRU ===
Cache size after adding 3 items: 3
Oldest key: b, Newest key: a
After adding 'd': a=1, b=None, c=3, d=4

=== Testing Statistics ===
Cache statistics: {'hits': 4, 'misses': 1, 'evictions': 1, 'insertions': 4, 'expirations': 0, 'current_size': 3, 'expired_count': 0}

=== Testing Cleanup ===
Before cleanup - size: 3
Expired items removed: 3, After cleanup - size: 0
